In [1]:
import gzip
from correctionlib.schemav2 import CorrectionSet,Correction,Category,CategoryItem,Formula
from itertools import chain
import correctionlib.schemav2 as schema
from JSONTools import *
import pandas as pd
import numpy as np

Welcome to JupyROOT 6.24/00


In [2]:
from SandS_class import *

In [3]:
#years=["2017","2018","2016preVFP","2016postVFP"]
years=["2016postVFP"]
folders=["2016postVFP_UL"]
gain=[1.0,6.0,12.0]

In [20]:
for year,folder in zip(years,folders):
    print(year)
    corrs=[]
    corr = Correction.parse_obj(
        {
            "version": 2,
            "name": f'ID',
            "description": "yes",
            "inputs": [
                {"name": "valtype","type": "string", "description": "correction (total_correction) or uncertainty (total_uncertainty)"},
                {"name": "gain","type": "real", "description": "seed gain"},
                {"name": "eta","type": "real", "description": "supercluster eta"},
                {"name": "r9", "type": "real", "description": "r9"},
                {"name": "et", "type": "real", "description": "et"},],
                #{"name": "gain", "type": "real", "description": "gain"},
            "output": {"name": "weight", "type": "real", "description": "value"},
            "data": Category.parse_obj({
                "nodetype": "category",
                "input": "valtype",
                "content": [
                    schema.CategoryItem.parse_obj({
                        "key": valtype, 
                        "value": Formula(
                    nodetype="formula",
                    variables=["gain","eta","r9","et"],
                    parser="TFormula",
                    expression="0+((x<2)*(y<2.5)*(z<0.5)*(t<100))",
                ),
                    })
                    for valtype in ["loose","tight"]
                ]
            })
        })
    corrs.append(corr)


    #Save JSON
    cset = CorrectionSet(schema_version=2, corrections=corrs,description=f"These are the photon ID, Pixel veto and CSEV Scale Factors (nominal, up or down) for {year} Ultra Legacy dataset. Each kind of scale factors, depend on the chosen working point, some other quantities. More information is below! Please keep in the mind that if your analysis is sensitive to high pT photons (>500 GeV), please refer https://twiki.cern.ch/twiki/bin/view/CMS/EGMPhotonIDHighPtPhotons")
    with open("ID.json", "w") as fout:
        fout.write(cset.json(exclude_unset=True, indent=4))

2016postVFP


In [21]:
from correctionlib import _core
#Download the correct JSON files
evaluator = _core.CorrectionSet.from_file('ID.json')

In [22]:
evaluator["ID"].evaluate("loose",1.0,2.0,0.2,99.0)

1.0

In [7]:
evaluator["2016postVFP_ScaleJSON"].evaluate("total_uncertainty",1,284044.0,-2.5,0.5,100.0)

0.03054039389999999

In [8]:
from Scale_simplepython import *

In [9]:
getsimplepythonScale("scale_nominal", "2016_postVFP", 284044, 2.5, 0.45, 100, 1)

{13001}


1.0180131299999997

In [10]:
getsimplepythonScale("scale_uncertainty", "2016_postVFP", 284044, 2.5, 0.45, 100, 1)

{13001}


0.03054039389999999